# Compute MFD for area sources

In [ ]:
# cleaner avoid to restart kernel for each code modification, use it just when alone
from cleaner import modules_cleaner ; modules_cleaner()
# load current project if secondary ipynb runned alone
import metys; metys.Metys.secondary()

In [ ]:
%matplotlib inline

import os
import re
import sys
import h5py
import numpy
import cPickle as pickle
from prettytable import PrettyTable
from hmtk.seismicity.occurrence.weichert import Weichert
from hmtk.seismicity.occurrence.utils import get_completeness_counts

In [ ]:
try:
    plot_figures
except: 
    plot_figures = False
    
try:
    print_log
except: 
    print_log = False

In [ ]:
if print_log or plot_figures:
    print '------------------------------------------------------'

# Width of bins used to discretise the MFDs
model_key = metys.g_prj.models.current 
binwidth = float(metys.g_prj.mod[model_key]['mfd_binwidth'])
if print_log:
    print 'MFD bin width    : %.2f' % (binwidth)

# Load sources
model_key = metys.g_prj.models.current 
data_path = os.path.join(metys.g_prj.folder, '%s_area_sources.pkl' % model_key)
fin = open(data_path,'rb') 
sources = pickle.load(fin)
fin.close()

# Check input variables
#if area_source_ids_list is None:
#    area_source_ids_list = ['10']
    
if len(area_source_ids_list) > 1:
    assert 0 == 1
    
src = sources[area_source_ids_list[0]]

In [ ]:
catalogue_filename = os.path.join(metys.g_prj.folder, '%s_catalogue_declustered.pkl' % model_key)
fin = open(catalogue_filename, 'rb') 
fcatal = pickle.load(fin)
fin.close()
if print_log:
    print 'The calogue contains %d earthquakes' % (len(fcatal.data['magnitude']))

In [ ]:
from mbt.tools.area import create_catalogue
fcatal = create_catalogue([src], fcatal, area_source_ids_list)

## Load catalogue completeness 

In [ ]:
compl_hdf5_filename = filename = os.path.join(metys.g_prj.folder, 'completeness.hdf5')
if print_log:
    print 'Reading', compl_hdf5_filename
fhdf5 = h5py.File(compl_hdf5_filename,'r')

grp = fhdf5[model_key]
src_id = area_source_ids_list[0]
if src_id in grp.keys():
    compl_table = grp[src_id][()]
    if print_log: 
        print 'Found completeness table for: <%s>' % (src_id)
else:
    if print_log:
        print 'hdf file does not contain completeness table for source: <%s>' % src_id
    if 'whole_catalogue' in grp.keys():
        compl_table = grp['whole_catalogue'][()]
        if looging:
            print 'using the default completeness table set for the whole catalogue'
    else:
        if print_log:
            print 'Default completeness table (whole catalogue) not defined'
        raise ValueError()
        compl_table = None
if print_log:
    print '\nCompleteness table'
    print compl_table
fhdf5.close()

## Computing GR double truncated

In [ ]:
idx = numpy.nonzero(compl_table[:,1] < numpy.max(fcatal.data['magnitude']))
weichert_config = {'magnitude_interval': 0.1, 
                   'reference_magnitude': 0.0}

weichert = Weichert()
bval_wei, sigmab, aval_wei, sigmaa = weichert.calculate(fcatal, weichert_config, 
                                                        completeness=compl_table)

if print_log:
    print 'bval: %.3f (sigma=%.3f)' % (bval_wei, sigmab)
    print 'aval: %.3f (sigma=%.3f)' % (aval_wei, sigmaa)

# Computing seismicity rates -----
cent_mag, t_per, n_obs = get_completeness_counts(fcatal, 
                                                 compl_table,
                                                 binwidth)

## Writing rates

In [ ]:
eqk_rates_hdf5_filename = os.path.join(metys.g_prj.folder, '%s_src_rates.pkl' % model_key)

if print_log:
    print 'Writing', eqk_rates_hdf5_filename
fhdf5 = h5py.File(eqk_rates_hdf5_filename,'a')
# Update/create group
if model_key in fhdf5.keys():
    if print_log:
        print 'Group exists. Set group %s' % (model_key)
    grp = fhdf5[model_key]
else:
    if print_log:
        print 'Create group: %s' % (model_key)
    grp = fhdf5.create_group(model_key)

# Update/create dataset
rates = numpy.array([cent_mag, t_per, n_obs])
if src_id in grp:
    del grp[src_id]
# Log info
if print_log:
    print 'Creating dataset %s' % (src_id)
dataset = grp.create_dataset(src_id, data=rates)
fhdf5.close()

## Plotting 

In [ ]:
if plot_figures: 
    from hmtk.plotting.seismicity.occurrence.recurrence_plot import plot_trunc_gr_model
    plot_trunc_gr_model(aval_wei, bval_wei, 4.0, 8.0, 0.1, catalogue=fcatal,
                        completeness=compl_table)

## Saving results

In [ ]:
src.a_gr=aval_wei
src.b_gr=bval_wei
sources[src.source_id] = src

output_path = os.path.join(metys.g_prj.folder, '%s_area_sources.pkl' % model_key)
asrc = {}
for key in sources.keys():
    src = sources[key]
    if src.source_type == 'AreaSource':
        asrc[key] = src    
fou = open(output_path,'wb') 
pickle.dump(asrc, fou)
fou.close()